### Generate all out

In [22]:
import os
import json
import soundfile as sf
import pyworld as pw
import matplotlib.pyplot as plt
import numpy as np
import soundfile as sf
import pyloudnorm as pyln
aiueo = ['a','i','u','e','o']

all_data = []
wav_dir = 'gliss_dataset'
out_dir = 'all_out'
note_step=0.25
frame_period = 10.0 # millisecond
out_dir = 'all_out'
fs = 44100
def get_f0_index_tbl(f0,min_note,max_note,note_step=0.25):
    f0_note = 12*np.log2(f0/440)+49
    out_len = int((max_note-min_note)/note_step+1)
    note_lin =np.linspace(min_note, max_note, num=out_len)
    idx_out = np.zeros(out_len,dtype = int)
    for i in range(out_len):
        idx_out[i] = np.argmin(abs(f0_note-note_lin[i]))
    return idx_out
meter = pyln.Meter(fs)
def loud_norm(x,db=-16.0):
    loudness = meter.integrated_loudness(x)
    return pyln.normalize.loudness(x, loudness, db)


In [52]:


if not os.path.exists(out_dir):
    os.makedirs(out_dir)

for who_i,who in enumerate(os.listdir(wav_dir)):  
    rangePath = os.path.join(wav_dir,who,"info.json")
    
    with open(rangePath, 'r') as f:
        info = json.load(f)
        [min_note,max_note] = info['pRange']
    yy = np.array([0])
    outpath = os.path.join(out_dir,who+'.wav')
    if os.path.isfile(outpath):
        print(who_i,who,'...done')
        continue
    print(min_note,max_note)
    
    for v in aiueo:
        path = os.path.join(wav_dir,who,v)
        print(path)
        x,fs = sf.read(path+'.wav')
        x = (x[:,0] + x[:,1])/2 
        x = x/max(x)*0.5
        f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)
        f0 = f0[1:-1]
        sp = sp[1:-1,:]
        ap = ap[1:-1,:]
        file_cnt = 2
        while os.path.isfile(path+str(file_cnt)+'.wav'):
            print(path+str(file_cnt)+'.wav')
            x2,_ = sf.read(path+str(file_cnt)+'.wav')
            x2 = (x2[:,0] + x2[:,1])/2 # stereo to mono
            x2 = x2/max(x2)*0.5
            f02, sp2,ap2 = pw.wav2world(x2, fs, frame_period =  frame_period)    # use default options

            f0 =np.concatenate((f0,f02[1:-1]), axis=0)
            sp =np.concatenate((sp,sp2[1:-1,:]), axis=0)
            ap = np.concatenate((ap,ap2[1:-1,:]), axis=0)
            file_cnt +=1
    #   remove f0 detect fail frame
        valid_idx = np.where(f0!=0)
        f0 =  f0[valid_idx]
        sp =  sp[valid_idx]
        ap =  ap[valid_idx]


        idx = get_f0_index_tbl(f0,min_note,max_note,note_step)  
        sp =  sp[idx,:]
        ap = ap[idx]

        lin_f0 = np.linspace(min_note, max_note,num=len(idx))
        f0 = 440*np.power(2,(lin_f0-49.0)/12.0)
    #     _y = pw.synthesize(f0, sp, ap[:len(idx)], fs, frame_period = 20)
        _y = pw.synthesize(f0, sp, ap, fs, frame_period = 20)

        yy = np.concatenate((yy,_y),axis = 0)

    
    sf.write(outpath, yy, fs)
print('done')

0 Abun ...done
1 boton ...done
2 cos ...done
3 LISA ...done
4 乙彤 ...done
5 偉晉 ...done
6 君豪 ...done
7 士峰 ...done
8 妤瑄 ...done
9 子齊 ...done
10 學煒 ...done
11 庭鈞 ...done
12 徐葳 ...done
13 志祥 ...done
14 我 ...done
15 敬恆 ...done
16 昱嘉 ...done
17 漢文 ...done
18 瀞云 ...done
19 繁傑 ...done
20 聖倫 ...done
21 贊全 ...done
22 長霖 ...done
done


In [65]:
x, fs = sf.read('a.wav')
x = (x[:,0] + x[:,1])/2 
x = x/max(x)*0.5
frame_period = 10.0 # millisecond
f0, sp, my_ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options
sp = sp[10,:]*np.ones((len(f0),1))
# f0 = f0[1]*np.ones((len(f0),1))[:,0]+np.random.randn(len(f0),)*0

y = pw.synthesize(f0,sp,my_ap, fs, frame_period = frame_period)
sf.write('./e_resynth.wav',y, fs)

### Generate sustain (juxtapose all output)

In [20]:
import os
import pickle
import soundfile as sf
import pyworld as pw
import numpy as np
import librosa

def note2f0(n):
    return 440.0*np.power(2,(n-49)/12.0)
gen_note = 35.5
gen_vowel = 'o'
gen_sec =  1.0;
fs = 44100;
d = pickle.load(open("./train_data/d.pkl", "rb"))
# idx = np.where(d[1]["note"] == 31.)
frame_period = 10.0 # millisecond
frames = np.int(gen_sec/frame_period*1000)
wav_lst = []
mel_basis = librosa.filters.mel(sr=fs, n_fft=2048, n_mels=13,fmin=0, fmax=fs / 2)
        
for i,ele in enumerate(d):
    seg_idx = np.where(ele[gen_vowel]['note'] == gen_note)[0]
    if len(seg_idx) != 0:
        print(i)
        sp = ele[gen_vowel]['sp'][seg_idx,:]*np.ones((frames,1)) 
        f0 = np.array([note2f0(gen_note)]*frames)+np.random.randn(frames,)
        ap = ele[gen_vowel]['ap'][seg_idx,:]*np.ones((frames,1))
        y = pw.synthesize(f0,sp,ap, fs, frame_period = frame_period)
        y = y/(max(y)-min(y))*0.4
        
        wav_lst.append(y)
        
print('done')

0
1
2
3
5
6
7
8
9
10
11
12
13
14
17
18
19
20
21
22
done


In [21]:
out = np.array([])
for wav_file in wav_lst:
    add_silence = np.concatenate((wav_file,np.zeros([4410,])))
    out = np.concatenate((out,add_silence))
sf.write('./sus_out/'+gen_vowel+'_'+str(gen_note)+'.wav', out, fs)
print('file write to','./sus_out/'+gen_vowel+'_'+str(gen_note)+'.wav')

file write to ./sus_out/o_35.5.wav


# Create Gender list and name list

In [14]:
g_lst = []
n_lst = []
for who_i,who in enumerate(os.listdir(wav_dir)):  
    rangePath = os.path.join(wav_dir,who,"info.json")
    with open(rangePath, 'r') as f:
        info = json.load(f)
        g_lst.append(info['gender'])
        n_lst.append(who)

# Gen sustain dir

In [13]:

for who_i,who in enumerate(os.listdir(wav_dir)):
    dir_name = './all_sustain/'+who
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)


# Generate all aiueo sustain 

In [29]:
import os
import pickle
import soundfile as sf
import pyworld as pw
import numpy as np
import librosa

def note2f0(n):
    return 440.0*np.power(2,(n-49)/12.0)
gen_note = 35.5
gen_vowel = 'o'
gen_sec =  1.0;
fs = 44100;
d = pickle.load(open("./train_data/d.pkl", "rb"))
frame_period = 10.0 # millisecond
frames = np.int(gen_sec/frame_period*1000)
# cnt = 0
for i,ele in enumerate(d):
    print(n_lst[i])
    for v in ['a','i','u','e','o']:
        seg_idx = np.where(ele[v]['note'] == gen_note)[0]
        if len(seg_idx) != 0:
            
            sp = ele[v]['sp'][seg_idx,:]*np.ones((frames,1)) 
            f0 = np.array([note2f0(gen_note)]*frames)+np.random.randn(frames,)*2
            ap = ele[v]['ap'][seg_idx,:]*np.ones((frames,1))
            y = pw.synthesize(f0,sp,ap, fs, frame_period = frame_period)
            y = y/(max(y)-min(y))*0.4
            out_path = './all_sustain/'+n_lst[i]+'/'+v+'.wav'
#             y = loud_norm(y,db = -20)
            sf.write(out_path,y, fs)
#     cnt+=1
#     if cnt == 2:
#         break
        
        
print('done')

Abun
boton
cos
Jenna
LISA
Namita
乙彤
亭鈞
偉晉
君豪
士峰
妤瑄
子齊
學煒
展延
徐葳
志祥
我
敬恆
昱嘉
漢文
瀞云
繁傑
done


In [5]:
sf.write('./test/foo.wav',y, fs)